In [6]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [7]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)

In [8]:
raw_documents = WikipediaLoader(query="Atelectasis").load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents) # should I only take the first? which is the most relevant?

In [9]:
db = Neo4jVector.from_documents(
    documents, OpenAIEmbeddings(), url=NOE4J_URL, username="neo4j", password="test1234"
)

retriever = db.as_retriever()

In [10]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

In [13]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"), chain_type="stuff", retriever=retriever
)

In [14]:
chain(
    {"question": "What is Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange. It is usually unilateral, affecting part or all of one lung. It is governed by English law.\n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis, 28-pl'}

In [17]:
chain(
    {"question": "What are the symptoms of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'The symptoms of Atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, it is a common misconception that atelectasis causes fever. There is no clinical evidence supporting this speculation. Atelectasis is often a common finding in chest X-rays and other radiological studies and may be caused by normal exhalation or various medical conditions. It is usually unilateral, affecting part or all of one lung. The most common cause of atelectasis is post-surgical atelectasis, which develops in 75-90% of people undergoing general anesthesia for a surgical procedure. Other causes include pulmonary tuberculosis, smoking, and age-related factors. The diagnosis of atelectasis is generally visible on chest X-ray, and treatment depends on the underlying cause. \n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasi

In [18]:
chain(
    {"question": "What can cause Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis can be caused by normal exhalation or by various medical conditions such as post-operative complications, surfactant deficiency, and infant respiratory distress syndrome. It can also be caused by factors such as bronchial obstruction, poor surfactant spreading, and suction. (',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis)'}

In [21]:
chain(
    {"question": "What clinical features radiologists need for diagnosing Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'For diagnosing Atelectasis, radiologists need to look for signs such as lung opacification, loss of lung volume, displacement of interlobar fissures, and mobile structures within the thorax. These features are generally visible on a chest X-ray. Additionally, clinical symptoms such as cough, chest pain, breathing difficulty, low oxygen saturation, pleural effusion, cyanosis, and increased heart rate may also be present. A CT scan or bronchoscopy may be necessary if the cause of atelectasis is not clinically apparent. \n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}

In [22]:
chain(
    {"question": "what are radiographic features of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Radiographic features of atelectasis include the collapse or closure of a lung resulting in reduced or absent gas exchange, unilateral involvement of part or all of one lung, and deflation of the alveoli. It is often seen in chest X-rays and other radiological studies. Atelectasis is not synonymous with pneumothorax, although it can cause atelectasis. It can be caused by normal exhalation or various medical conditions. Acute atelectasis may occur as a post-operative complication or due to surfactant deficiency. In premature babies, it can lead to infant respiratory distress syndrome. \n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}